# 🤖 Assistente Virtual Híbrido (IA + Comandos Locais)

Este notebook apresenta a implementação de um assistente virtual modular. Ele combina o poder de modelos de linguagem de larga escala (LLMs) com funcionalidades locais de automação.

### 🌟 Funcionalidades Principais:
- **STT (Speech to Text)**: Reconhecimento de fala local utilizando o modelo **OpenAI Whisper**.
- **TTS (Text to Speech)**: Síntese de voz offline através da biblioteca **pyttsx3**.
- **IA (Cérebro)**: Integração com o modelo **GLM-4.7-Flash** via Hugging Face Router.
- **Automação Local**: Comandos diretos para Wikipedia, YouTube e busca por serviços próximos (ex: farmácias).

## 🛠️ Passo 1: Instalação de Dependências

Antes de começar, precisamos garantir que todas as bibliotecas necessárias estejam instaladas no ambiente. Este bloco detecta dependências ausentes e as instala automaticamente.

In [1]:
# Verificação e instalação automática de dependências
try:
    # Tenta importar as bibliotecas principais para checar se existem
    import openai
    import whisper
    import sounddevice
    import pyttsx3
    import dotenv
    print("✅ Dependências já estão configuradas no ambiente.")
except ImportError:
    # Caso alguma falhe, inicia a instalação via pip
    print("⏳ Algumas dependências não foram encontradas. Instalando... (isso pode levar alguns minutos)")
    # Bibliotecas:
    # - openai: Interface para a API da IA
    # - openai-whisper: Modelo de transcrição de áudio local
    # - sounddevice & scipy: Captura e processamento de áudio do microfone
    # - pyttsx3: Motor de síntese de voz offline
    # - python-dotenv: Carregamento de chaves de API de arquivos .env
    %pip install openai openai-whisper sounddevice scipy pyttsx3 python-dotenv

✅ Dependências já estão configuradas no ambiente.


## ⚙️ Passo 2: Importações e Configurações

Importamos os módulos do Python e carregamos as variáveis de ambiente (como o token do Hugging Face).

In [2]:
import argparse
from typing import Protocol, Optional, Iterable
from dataclasses import dataclass
import os
import urllib.parse
import webbrowser
import time
from dotenv import load_dotenv

# Carrega as configurações do arquivo .env (Token da API, etc.)
load_dotenv()

print("✅ Módulos importados e configurações de ambiente carregadas.")

✅ Módulos importados e configurações de ambiente carregadas.


## 🏗️ Passo 3: Definição de Interfaces (Protocolos)

Para manter o código organizado e modular, definimos interfaces para o Reconhecimento de Voz (STT) e Síntese de Voz (TTS). Isso permite trocar as tecnologias sem alterar a lógica principal do assistente.

In [3]:
class SpeechToText(Protocol):
    """Interface para componentes que convertem fala em texto."""
    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        """Ouve o áudio e retorna a string transcrita ou None."""
        pass

class TextToSpeech(Protocol):
    """Interface para componentes que convertem texto em fala."""
    def speak(self, text: str) -> None:
        """Processa a string de texto e emite o som correspondente."""
        pass

## 🎙️ Passo 4: Implementação do STT (Escuta)

Implementamos duas formas de entrada: 
1. **WhisperSTT**: Usa o microfone e o modelo local do Whisper.
2. **TextInputSTT**: Permite interagir via teclado (ideal para depuração ou ambientes sem microfone).

In [4]:
class WhisperSTT:
    """Reconhecimento de fala local usando OpenAI Whisper."""
    def __init__(self, model_size: str = "base", language: str = "pt", duration: int = 5):
        """
        Inicializa o modelo Whisper.
        :param model_size: Tamanho do modelo (base, tiny, small, etc.)
        :param language: Idioma para transcrição
        :param duration: Tempo padrão de gravação em segundos
        """
        # Importações internas para carregar apenas se necessário
        import whisper
        import sounddevice as sd
        import scipy.io.wavfile as wav
        import tempfile
        import os
        
        self._whisper = whisper
        self._sd = sd
        self._wav = wav
        self._tempfile = tempfile
        self._os = os
        
        # Carrega o modelo na memória
        print(f"⏳ Carregando modelo Whisper '{model_size}'...")
        self._model = whisper.load_model(model_size)
        self._language = language
        self._duration = duration

    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        """Grava o microfone e transcreve o áudio para texto."""
        duration = timeout if timeout is not None else self._duration
        fs = 44100  # Frequência de amostragem
        
        print(f"\n🎤 [Escutando...] Fale agora ({duration}s).")
        
        try:
            # Grava áudio do dispositivo de entrada padrão
            recording = self._sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
            self._sd.wait()  # Espera a gravação terminar
            print("⏳ [Processando...]")
            
            # Salva em um arquivo temporário para processamento do Whisper
            with self._tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
                temp_filename = f.name
            
            self._wav.write(temp_filename, fs, recording)
            
            # Transcrição
            result = self._model.transcribe(temp_filename, language=self._language, fp16=False)
            text = result["text"].strip()
            
            # Limpeza do arquivo temporário
            try:
                self._os.remove(temp_filename)
            except:
                pass
                
            return text if text else None
        except Exception as e:
            print(f"❌ Erro no processamento de áudio: {e}")
            return None

class TextInputSTT:
    """Simula reconhecimento de fala através de entrada de texto no console."""
    def __init__(self, inputs: Optional[Iterable[str]] = None):
        self._inputs = list(inputs) if inputs is not None else None

    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        """Lê do teclado ou de uma lista pré-definida de comandos."""
        if self._inputs is not None:
            if not self._inputs: return None
            return self._inputs.pop(0)
        try:
            return input("\n⌨️ Digite seu comando (ou 'sair'): ").strip()
        except EOFError:
            return None

## 🔊 Passo 5: Implementação do TTS (Fala)

Configuramos a saída de voz. O **Pyttsx3TTS** utiliza as vozes instaladas no sistema operacional, funcionando totalmente offline.

In [5]:
class Pyttsx3TTS:
    """Síntese de voz offline usando pyttsx3."""
    def __init__(self, language: str = "pt-BR", rate: Optional[int] = None):
        """
        Inicializa o motor de voz.
        :param language: Prefixo do idioma (ex: pt)
        :param rate: Velocidade da fala
        """
        import pyttsx3
        self._engine = pyttsx3.init()
        self._language = language
        
        if rate is not None:
            self._engine.setProperty("rate", rate)
            
        self._select_voice()

    def _select_voice(self) -> None:
        """Busca no sistema uma voz compatível com o idioma escolhido."""
        voices = self._engine.getProperty("voices")
        chosen = None
        for v in voices:
            name = getattr(v, "name", "") or ""
            lang = "".join(getattr(v, "languages", []) or [])
            # Verifica se o idioma ou nome da voz contém o código do idioma (ex: pt)
            if self._language.lower()[:2] in (lang.lower(), name.lower()):
                chosen = v.id
                break
        if chosen:
            self._engine.setProperty("voice", chosen)

    def speak(self, text: str) -> None:
        """Imprime o texto e reproduz o áudio."""
        print(f"🤖 Assistente: {text}")
        try:
            self._engine.say(text)
            self._engine.runAndWait()
        except Exception as e:
            print(f"⚠️ Erro ao emitir som: {e}")

class SilentTTS:
    """Apenas imprime as respostas, sem emitir som."""
    def speak(self, text: str) -> None:
        print(f"🤖 Assistente (Modo Silencioso): {text}")

## 🧠 Passo 6: Integração com Inteligência Artificial

Conectamos o assistente ao modelo **GLM-4.7-Flash**. Esta função inclui lógica de retentativa para garantir robustez contra falhas temporárias de rede.

In [6]:
def get_glm_response(text: str) -> Optional[str]:
    """
    Envia o texto para a API da IA e retorna a resposta gerada.
    Inclui 3 tentativas em caso de erro de timeout ou servidor.
    """
    max_retries = 3
    retry_delay = 2
    
    for attempt in range(max_retries):
        try:
            from openai import OpenAI
            
            # Recupera o token do arquivo .env
            hf_token = os.getenv("HF_TOKEN")
            if not hf_token or hf_token == "seu_token_hf_aqui":
                return "⚠️ Erro: HF_TOKEN não configurado no arquivo .env."
                
            # Configura o cliente para o Router do Hugging Face
            client = OpenAI(
                base_url="https://router.huggingface.co/v1",
                api_key=hf_token,
                timeout=30.0
            )
            
            # Chamada de chat completion
            response = client.chat.completions.create(
                model="zai-org/GLM-4.7-Flash",
                messages=[
                    {"role": "system", "content": "Você é um assistente virtual prestativo e conciso. Responda em português brasileiro."},
                    {"role": "user", "content": text}
                ]
            )
            return response.choices[0].message.content
            
        except Exception as e:
            error_str = str(e)
            # Verifica se é um erro que vale a pena tentar de novo (502, 503, 504, Timeout)
            if any(code in error_str for code in ["504", "502", "503", "timeout"]) and attempt < max_retries - 1:
                print(f"⚠️ Falha na conexão com a IA. Tentando novamente em {retry_delay}s... (Tentativa {attempt + 1})")
                time.sleep(retry_delay * (attempt + 1))
                continue
            
            # Trata erros específicos de infraestrutura (respostas HTML)
            if "<!DOCTYPE html>" in error_str or "<html>" in error_str:
                return "❌ O servidor de IA está instável no momento. Por favor, tente novamente mais tarde."
                
            return f"❌ Erro técnico na IA: {error_str}"
            
    return "❌ Falha definitiva após múltiplas tentativas de conexão."

## ⚡ Passo 7: Processamento de Comandos e Ações

Aqui definimos a lógica que decide se o comando do usuário é um atalho local (abrir site, pesquisar) ou se deve ser enviado para a IA.

In [7]:
@dataclass
class ActionResult:
    """Classe simples para armazenar o resultado de uma execução de comando."""
    success: bool
    message: str
    is_ai: bool = False

def parse_and_execute(text: str) -> ActionResult:
    """
    Analisa a string de entrada e executa a ação correspondente.
    Prioriza comandos locais antes de enviar para a IA.
    """
    s = (text or "").lower().strip()
    if not s: return ActionResult(False, "Não entendi o que você disse.")
    
    # --- COMANDOS LOCAIS ---
    
    # Ação: Pesquisar na Wikipedia
    if "wikipedia" in s:
        q = s.replace("wikipedia", "").replace("pesquisar", "").strip()
        url = "https://pt.wikipedia.org/wiki/Special:Search?search=" + urllib.parse.quote_plus(q)
        webbrowser.open(url)
        return ActionResult(True, f"Pesquisando sobre '{q}' na Wikipedia.")
        
    # Ação: Pesquisar no YouTube
    if "youtube" in s or "video" in s:
        q = s.replace("youtube", "").replace("video", "").replace("pesquisar", "").strip()
        url = "https://www.youtube.com/results?search_query=" + urllib.parse.quote_plus(q)
        webbrowser.open(url)
        return ActionResult(True, f"Buscando vídeos sobre '{q}' no YouTube.")
        
    # Ação: Buscar farmácia próxima
    if "farmácia" in s or "farmacia" in s:
        webbrowser.open("https://www.google.com/maps/search/farmacia+perto+de+mim")
        return ActionResult(True, "Abrindo o mapa com as farmácias mais próximas.")
    
    # --- PROCESSAMENTO VIA IA ---
    # Se nenhum comando local foi detectado, pergunta ao GLM-4
    print("🔍 Consultando a inteligência artificial...")
    ai_response = get_glm_response(text)
    
    if ai_response:
        return ActionResult(True, ai_response, is_ai=True)
        
    return ActionResult(False, "Desculpe, não consegui processar seu pedido.")

## 🕹️ Passo 8: Classe Orquestradora do Assistente

Esta classe gerencia o ciclo de vida do assistente: ouvir o usuário, processar a intenção e responder via áudio.

In [8]:
class Assistant:
    """Gerenciador principal do fluxo de conversação do assistente virtual."""
    def __init__(self, stt: SpeechToText, tts: TextToSpeech):
        """
        Injeta as dependências de Voz e Fala.
        :param stt: Instância de um provedor de fala-para-texto
        :param tts: Instância de um provedor de texto-para-fala
        """
        self._stt = stt
        self._tts = tts

    def run(self):
        """Inicia o loop infinito de interação."""
        self._tts.speak("Olá! Sou seu assistente virtual inteligente. Como posso ajudar você hoje?")
        
        while True:
            # 1. Escuta a entrada do usuário
            text = self._stt.listen()
            if not text: continue
            
            print(f"👤 Usuário: {text}")
            
            # 2. Verifica se o usuário quer encerrar o programa
            if text.lower().strip() in ["sair", "encerrar", "tchau", "finalizar", "adeus"]:
                self._tts.speak("Entendido. Encerrando o sistema. Até logo!")
                break
                
            # 3. Processa o comando (Local ou IA)
            result = parse_and_execute(text)
            
            # 4. Responde via áudio e texto
            self._tts.speak(result.message)

## 🚀 Passo 9: Inicialização e Execução

Agora, configuramos as implementações que queremos usar (Voz vs Teclado) e iniciamos o assistente.

In [9]:
# Configuração dos componentes para o Notebook

# Recomendado para Notebook: Entrada via teclado (TextInputSTT)
# Para usar microfone, descomente a linha do WhisperSTT abaixo e comente a do TextInputSTT
stt = TextInputSTT()
# stt = WhisperSTT(duration=5)

try:
    # Tenta carregar o motor de voz local
    tts = Pyttsx3TTS()
except Exception as e:
    # Se o sistema não tiver suporte a som, usa o modo silencioso (apenas texto)
    print(f"⚠️ Aviso: Não foi possível iniciar o motor de voz ({e}). Usando modo silencioso.")
    tts = SilentTTS()

# Instancia e executa o assistente
assistant = Assistant(stt, tts)
assistant.run()

🤖 Assistente: Olá! Sou seu assistente virtual inteligente. Como posso ajudar você hoje?
👤 Usuário: oi
🔍 Consultando a inteligência artificial...
🤖 Assistente: Olá! Como posso ajudar você hoje?
👤 Usuário: sair
🤖 Assistente: Entendido. Encerrando o sistema. Até logo!
